In [2]:
import numpy as np
import pandas as pd
from multiprocessing import Pool
import itertools

In [3]:
pndf = pd.read_csv('pndf.csv')
res = pd.read_csv('res 340.csv')

In [ ]:
import numpy as np
import pandas as pd
from multiprocessing import Pool
import itertools


#scoring function
def calculate_element_score3(match_award, mismatch_penalty, gap_penalty, seq1, seq2):
    m, n = len(seq1), len(seq2)  # length of two sequences

    # Generate DP table and traceback path pointer matrix
    score = np.zeros((m+1, n+1)).astype(int)      # the DP table
    
    max_score = 0        # initial maximum score in DP table
    # Calculate DP table

    scores_list = []

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            score_diagonal = score[i-1][j-1] + match_score(seq1[i-1], seq2[j-1])
            score_up = score[i][j-1] + gap_penalty
            score_left = score[i-1][j] + gap_penalty
            score[i][j] = max(0,score_left, score_up, score_diagonal)
            if score[i][j] >= max_score:
                max_i = i
                max_j = j
                max_score = score[i][j];

    perfect_score = min(len(seq1), len(seq2)) * match_award

    perfect_matches = np.count_nonzero(score == perfect_score)

    first_tolerance_match = np.count_nonzero(score == perfect_score - 1)
    second_tolerance_match = np.count_nonzero(score == perfect_score - 2)
    third_tolerance_match = np.count_nonzero(score == perfect_score - 3)

    final_score = perfect_matches + first_tolerance_match/(perfect_score * 2) + second_tolerance_match/(perfect_score * 4) + third_tolerance_match/(perfect_score * 8)
    
    return final_score

def match_score(alpha, beta):
    if alpha == beta:
        return match_award
    elif alpha == '-' or beta == '-':
        return gap_penalty
    else:
        return mismatch_penalty
    
#set other parameters
match_award      = 2
mismatch_penalty = -1
gap_penalty      = -1 # both for opening and extanding

#s is an individual sequence handed to the function
#score is returned as a numerical value

def f(s):
    
    with open('temp.txt', 'r') as file2:
        ele = file2.readline()
                
    score = calculate_element_score3(match_award, mismatch_penalty, gap_penalty, ele, s)

    return score



if __name__ == '__main__':
    
    
    #load data
    
    pndf = pd.read_csv('pndf.csv')
    df = pndf

    dfp = df.loc[df['Gen'] == 'ssRNA(+)']
    dfn = df.loc[df['Gen'] == 'ssRNA(-)']

    dfp = dfp.reset_index(drop = True)
    dfn = dfn.reset_index(drop = True)
    
    
    #make windows

    winlist6 = []

    for i in range(6,7):
        for i in (itertools.product('ATGC', repeat=i)):
            winlist6.append(''.join(i))
            
            
    winlist = ['A', 'TC', 'TGA', 'TGGT', 'ATC']
    
    
    #working dataframe and result df 
    df = pd.concat([dfp,dfn])
    resultdf = pd.DataFrame(index = list(range(len(df))), columns = winlist)
          
    #working loop
    
    for element in winlist:
        pool = Pool(processes=4)
        
        with open('temp.txt', 'w') as file:
            file.write(element)
            
        it = iter(df['Seq'].to_list())
        
        #result returned as a list the length of iterator it
        result = pool.map(f, it)
                
        resultdf[element] = result
        
        pool.close()
        
        #writing in 3 parts
        if winlist.index(element) == 1024:
            resultdf.to_csv('win6 p1.csv', index = False)
        elif winlist.index(element) == 2048:
            resultdf.to_csv('win6 p2.csv', index = False)
        elif winlist.index(element) == 3072:
            resultdf.to_csv('win6 p3.csv', index = False)
                
    resultdf.to_csv('win tdd .csv', index = False)

In [10]:
a = pd.read_csv('resultp-window-6-upto-200.csv')

In [11]:
a

,AAAAAA,AAAAAT,AAAAAG,AAAAAC,AAAATA,AAAATT,AAAATG,AAAATC,AAAAGA,AAAAGT,...,AAGCCG,AAGCCC,AACAAA,AACAAT,AACAAG,AACAAC,AACATA,AACATT,AACATG,AACATC
0,43.000000,37.270833,40.416667,33.781250,33.385417,41.000000,44.218750,26.635417,43.041667,36.635417,...,3.989583,5.802083,50.697917,51.250000,31.281250,44.614583,30.458333,42.697917,36.802083,23.812500
1,31.604167,12.302083,14.281250,12.291667,14.906250,12.447917,16.145833,9.260417,14.531250,9.666667,...,3.114583,3.770833,19.354167,16.489583,12.593750,13.812500,8.729167,13.739583,15.208333,9.031250
2,11.354167,14.135417,13.843750,9.927083,10.031250,16.239583,13.437500,8.052083,19.989583,11.958333,...,5.468750,3.552083,15.375000,18.864583,16.520833,7.531250,5.677083,13.072917,8.677083,8.416667
3,33.239583,17.302083,15.833333,11.291667,13.010417,24.187500,21.583333,14.802083,13.916667,14.375000,...,6.906250,6.427083,12.645833,19.791667,12.218750,10.593750,12.656250,21.322917,17.260417,15.739583
4,23.395833,20.937500,17.781250,14.489583,18.802083,12.020833,8.395833,10.052083,19.906250,12.562500,...,4.239583,12.416667,14.062500,15.135417,18.187500,15.375000,11.177083,6.770833,9.666667,5.718750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2857,2.041667,2.000000,4.656250,1.916667,3.114583,2.166667,3.083333,4.166667,7.333333,4.510417,...,6.375000,7.510417,3.010417,5.656250,6.260417,10.416667,3.197917,8.552083,5.885417,3.552083
2858,1.031250,2.104167,6.520833,3.208333,0.937500,0.864583,5.645833,3.500000,3.708333,3.854167,...,2.520833,2.229167,1.406250,1.739583,8.322917,2.645833,1.375000,1.572917,4.437500,9.270833
2859,26.635417,33.822917,17.968750,21.718750,17.562500,12.312500,15.250000,15.760417,26.531250,10.583333,...,3.937500,6.916667,19.895833,9.031250,12.843750,13.406250,13.072917,3.875000,11.447917,6.750000
2860,1.145833,2.020833,1.895833,1.177083,3.968750,3.354167,0.875000,1.156250,1.114583,1.979167,...,2.010417,0.822917,0.687500,0.625000,0.541667,1.937500,0.885417,1.916667,0.822917,0.937500


In [18]:
a = a.div(pndf['Len'],axis = 0)

In [19]:
a

,A,TC,TGA,TGGT,ATC
0,0.537206,0.110160,0.040584,0.013060,0.028280
1,0.570012,0.105699,0.038106,0.009164,0.034410
2,0.548775,0.109577,0.044159,0.009273,0.031324
3,0.495611,0.115402,0.043164,0.016777,0.027276
4,0.548424,0.088494,0.038791,0.010999,0.025641
...,...,...,...,...,...
3140,0.545363,0.122817,0.039605,0.008120,0.040939
3141,0.531312,0.122658,0.038540,0.007008,0.039075
3142,0.532276,0.118986,0.038647,0.007257,0.040820
3143,0.532001,0.119003,0.038631,0.007335,0.040771


In [21]:
res[a.columns]

,A,TC,TGA,TGGT,ATC
0,0.537206,0.110160,0.040584,0.013060,0.028280
1,0.570012,0.105699,0.038106,0.009164,0.034410
2,0.548775,0.109577,0.044159,0.009273,0.031324
3,0.495611,0.115402,0.043164,0.016777,0.027276
4,0.548424,0.088494,0.038791,0.010999,0.025641
...,...,...,...,...,...
3140,0.545363,0.122817,0.039605,0.008120,0.040939
3141,0.531312,0.122658,0.038540,0.007008,0.039075
3142,0.532276,0.118986,0.038647,0.007257,0.040820
3143,0.532001,0.119003,0.038631,0.007335,0.040771
